In [1]:

import numpy as np
import matplotlib.pyplot as plt
import sys
import os
from tqdm import tqdm
sys.path.append('..')

In [2]:

import numpy as np
import matplotlib.pyplot as plt
import sys
import os
from tqdm import tqdm
sys.path.append('..')

In [3]:
%load_ext autoreload
%autoreload 2
from solat_cb.simulation import *
from solat_cb.spectra import *
from solat_cb.mle import *

In [4]:
libdir ='/pscratch/sd/l/lonappan/SOLAT'
nside = 1024
cb_method = 'iso'
beta = 0.35
dust = 10
synch = 5
alpha = [-0.1,-0.1,0.2,0.2,.15,.15]
atm_noise = True
nsplits = 2
bandpass = False
fit = "As + Asd + Ad + beta + alpha"
binwidth = 20
bmin = 60
bmax = 2000

In [5]:
lat = LATsky(libdir,nside,cb_method,dust,synch,alpha,beta,atm_noise=atm_noise,nsplits=nsplits,bandpass=bandpass)

CMB : INFO - Loading CMB power spectra from file
Noise : INFO - Noise Model: White + 1/f noise v3.0.0


In [6]:
spec = Spectra(lat,cache=True,parallel=0,dust_model=9,sync_model=4)

Spectra : WARNING - Evaluating special case: Simulation uses 'd10s5' FG model
Spectra : WARNING - The template foreground is set to d9s4
Spectra : INFO - Reading apodised mask from /pscratch/sd/l/lonappan/SOLAT/LAT_atm_noise_2splits/spectra_1024_aposcale2p0_common_temp94/workspaces/mask_N1024_aposcale2p0_CO_PS.fits
Spectra : INFO - Reading coupling Matrix from /pscratch/sd/l/lonappan/SOLAT/LAT_atm_noise_2splits/spectra_1024_aposcale2p0_common_temp94/workspaces/coupling_matrix_N1024_fsky0p55_aposcale2p0_CO_PS.fits
Spectra : INFO - No parallelization


In [7]:
ml = MLE(libdir,spec,fit, alpha_per_split=False,rm_same_tube=False,binwidth=binwidth,bmin=bmin,bmax=bmax)

MLE : WARNING - Special Case Noted: FG model in LATsky and Spectra object are different
CMB : INFO - Loading CMB power spectra from file


Fitting a common polarisation angle per frequency


In [8]:
ml.estimate_angles(0)

Spectra : WARNING - Special case: Assumes that there exsist a previous run
Spectra : WARNING - Special case: the default obsxobs directory is /pscratch/sd/l/lonappan/SOLAT/LAT_atm_noise_2splits/spectra_1024_aposcale2p0_atm_noise_temp94/obs_x_obs
Spectra : WARNING - Special case: the obsxobs directory is set to /pscratch/sd/l/lonappan/SOLAT/LAT_atm_noise_2splits/spectra_1024_aposcale2p0_atm_noise/obs_x_obs
Spectra : WARNING - Special case: Assumes that there exsist a previous run
Spectra : WARNING - Special case: the default obsxobs directory is /pscratch/sd/l/lonappan/SOLAT/LAT_atm_noise_2splits/spectra_1024_aposcale2p0_atm_noise_temp94/obs_x_obs
Spectra : WARNING - Special case: the obsxobs directory is set to /pscratch/sd/l/lonappan/SOLAT/LAT_atm_noise_2splits/spectra_1024_aposcale2p0_atm_noise/obs_x_obs
Spectra : WARNING - Special case: Assumes that there exsist a previous run
Spectra : WARNING - Special case: the default obsxobs directory is /pscratch/sd/l/lonappan/SOLAT/LAT_atm_no

{'As': 2.305363478408839,
 'Asd': 0.9038933948687158,
 'Ad': 0.8887309021443616,
 'beta': 0.28402656574056373,
 '27': 0.04516888796821087,
 '39': 0.057727178037557184,
 '93': 0.25831250490404073,
 '145': 0.25959069501893894,
 '225': 0.17709372961815065,
 '280': 0.17659626332156558}

In [9]:
LowStat(ml).estimated_params()

{'beta': {'mean': 0.3580033931365695, 'std': 0.1025170371617966},
 '27': {'mean': -0.11006686140310261, 'std': 0.11648512683316616},
 '39': {'mean': -0.11181359064332706, 'std': 0.11244022513745283},
 '93': {'mean': 0.1910102275421927, 'std': 0.10251551352074266},
 '145': {'mean': 0.1901989988166308, 'std': 0.10024022845119114},
 '225': {'mean': 0.13999312290185553, 'std': 0.1009310144204663},
 '280': {'mean': 0.14385157823123215, 'std': 0.10279051103692242}}